In [1]:
import decimal

In [2]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [5]:
decimal.getcontext().prec = 14
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [6]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [7]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(decimal.Decimal(1) / decimal.Decimal(3))
decimal.getcontext().prec = old_prec
print(decimal.Decimal(1) / decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [8]:
class precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec
        
    def __enter__(self):
        decimal.getcontext().prec = self.prec
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False

In [9]:
with precision(3):
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [11]:
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [12]:
from time import perf_counter, sleep
class Timer:
    def __init__(self):
        self.elapsed = 0
        
    def __enter__(self):
        self.start = perf_counter()
        return self
    
    def __exit__(self, a, b, c):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [15]:
with Timer() as timer:
    sleep(1)
    
print(timer.elapsed)

0.9999265999999807


In [16]:
print('hello')

hello


In [17]:
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout
        
    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file
        
    def __exit__(self, a, b, c):
        sys.stdout = self._current_stdout
        self._file.close()
        return False
    

In [21]:
with OutToFile('test.txt'):
    print(sys.stdout)
    print('Line 1')
    print('Line 2')

In [22]:
print('Line 1')

Line 1


In [23]:
sys.stdout

In [24]:
with open('test.txt') as f:
    print(f.readlines())

["<_io.TextIOWrapper name='test.txt' mode='w' encoding='cp1252'>\n", 'Line 1\n', 'Line 2\n']


In [25]:
print('Line 1')

Line 1


In [26]:
class Tag:
    def __init__(self, tag):
        self._tag = tag
        
    def __enter__(self):
        print(f'<{self._tag}>', end='')
        
    def __exit__(self, a, b, c):
        print(f'/<{self._tag}>', end='')
        return False

In [27]:
with Tag('p'):
    print('some ', end='')
    with Tag('b'):
        print('bold', end='')
    print(' text', end='')

<p>some <b>bold/<b> text/<p>

In [30]:
class ListMaker:
    def __init__(self, title, prefix='- ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)
        
    def __enter__(self):
        self._current_indent += self._indent
        return self
    
    def __exit__(self, a, b, c):
        self._current_indent -= self._indent
        return False
    
    def print_item(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        print(s)

In [33]:
with ListMaker('Items') as lm:
    lm.print_item('Item 1')
    with lm:
        lm.print_item('sub item 1a')
        lm.print_item('sub item 1b')
        with lm:
            lm.print_item('******')
    lm.print_item('Item 2')
    with lm:
        lm.print_item('sub item 2a')
        lm.print_item('sub item 2b')
    

Items
   - Item 1
      - sub item 1a
      - sub item 1b
         - ******
   - Item 2
      - sub item 2a
      - sub item 2b


In [35]:
with OutToFile('my_list.txt'):
    with ListMaker('Items') as lm:
        lm.print_item('Item 1')
        with lm:
            lm.print_item('sub item 1a')
            lm.print_item('sub item 1b')
            with lm:
                lm.print_item('******')
        lm.print_item('Item 2')
        with lm:
            lm.print_item('sub item 2a')
            lm.print_item('sub item 2b')


In [38]:
with open('my_list.txt') as f:
    for row in f:
        print(row, end='')

Items
   - Item 1
      - sub item 1a
      - sub item 1b
         - ******
   - Item 2
      - sub item 2a
      - sub item 2b
